In [749]:
from pymongo import MongoClient
client = MongoClient('mongodb://10.120.30.8:27017')
db = client['music']  
collect = db['mscx_c_key']


In [859]:
#一次抓4個
#修改txtfilePath內容，再執行

from operator import itemgetter
import csv,os

txtfilePath = b'E:/downloadmusic4/'
csvfilePath = b'C:/Users/BigData/'
Dominanatvalue = ['G', 'Am', 'Bb', 'C', 'F', 'Dm', 'Em']

def Dominantlist_from_txt(txtPath):
    with open(txtPath, 'rb') as txtFile:
        txtreadline = txtFile.readline()
    txtreadline = txtreadline.split(',')
    Dominantlist = []
    for i in range(len(txtreadline)):
        if txtreadline[i] == '0':
            Dominantlist.append(int(txtreadline[i]))
        else:
            Dominantlist.append(txtreadline[i])
    return Dominantlist

def Dominantlist_to_Dominantgram_4freq(Dominantlist):
    return [Dominantlist[i:i+4] for i in range(len(Dominantlist)-3) if (i%2)==0]

def Dominantgram_to_Dominantdict_4freq(Dominantgram):
    Dominantdict = dict()
    for i in range(len(Dominantgram)):
        if Dominantgram[i][0] != 0 :
            Dominantdict[tuple(Dominantgram[i])] = Dominantdict.get(tuple(Dominantgram[i]),0)+1
    return Dominantdict

def Dominantsorted_count_comparator(x):
    return x[1]

#想像[1~4]，每次往右位移2個，變成[3,4,1~2]，這些可能的變化，都不計算
def Dominantsorted_possible_transfer_clean(Dominantsorted):
    lists = []
    for i in range(len(Dominantsorted)):
        k = 2
        splitcount = 0
        transferlists = []
        for j in range(len(Dominantsorted[0][0])):
            if j%2==0 and j!=0 :
                splitcount += 1
                transferlists.append(Dominantsorted[i][0][(k*(splitcount)):] + Dominantsorted[i][0][:(k*(splitcount))])
        if set(transferlists).isdisjoint(set(lists)) :
            lists.append(Dominantsorted[i][0])
    return lists

def Dominantsorted_classify(Compare_list,Compare_number):
    Compare_list2 = []
    print '用來比對的和弦'
    print Compare_list[Compare_number]
    print '和弦組合當中，類似的組合'
    for i in range(len(Compare_list)):
        Compare_count = 0
        for j in range(len(Compare_list[0])):
            if cmp(Compare_list[Compare_number][j],Compare_list[i][j]) == 0 : Compare_count += 1       
        print Compare_count,Compare_list[i]
        if Compare_count >= 3 : Compare_list2.append(Compare_list[i])
    return Compare_list2

#目前一次四個和弦，最多錯一個和弦，然後去掉重複的，做成list，用來比對
def Dominantsorted_to_Dominantcompare_4freq(Dominantsorted,Compare_number):
    Compare_list = Dominantsorted_possible_transfer_clean(Dominantsorted)
    Compare_Dominant = [Dominantsorted[i] for i in range(len(Dominantsorted)) for j in Compare_list if Dominantsorted[i][0] == j]
    print '可用來比對的和弦組合'
    print Compare_Dominant
    print '總共有幾個和弦組合可用來比對 =',len(Compare_list)
    Compare_list2 = Dominantsorted_classify(Compare_list,Compare_number)
    print '挑選的結果'
    Compare_list3 = [j for j in  Dominantsorted for i in Compare_list2 if j[0] == i]
    print Compare_list3
    print '歸類到最大次數的和弦'
    Compare_list3 = max(Compare_list3,key = Dominantsorted_count_comparator)
    print Compare_list3

for txtfilename in os.listdir(txtfilePath):
    print '檔名 = ' + txtfilename.split('.')[0]
    txtPath = txtfilePath + txtfilename
    Dominantlist = Dominantlist_from_txt(txtPath)
#     print Dominantlist
    Dominantgram_4freq = Dominantlist_to_Dominantgram_4freq(Dominantlist)
#     print Dominantgram_4freq
    Dominantdict_4freq = Dominantgram_to_Dominantdict_4freq(Dominantgram_4freq)
#     print Dominantdict_4freq
    Dominantsorted_4freq=sorted(Dominantdict_4freq.items(), key=itemgetter(1), reverse=True)
#     print Dominantsorted_4freq
#     這裡可以改比對的參數，以第幾個去比對
    Dominantsorted_to_Dominantcompare_4freq(Dominantsorted_4freq,1)
    

#寫入csv檔
csvfilePath = b'C:/Users/BigData/4-gram.csv'
# csvPath = csvfilePath + unicode_mscxfilename + ' 4-gram.csv'

with open(csvfilePath,'wb') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['Dominant', 'Frequency'])
    for i in range(0,len(Dominantsorted_4freq)):
        writer.writerow(Dominantsorted_4freq[i])

檔名 = Alone-in-my-room-test
[0, 0, 0, 0, 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bb', 'C', 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bdim', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'F', 'Bb', 'Em', 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bb', 'C', 'F', 'Am', 'Dm', 'Am', 'F', 'Am', 'Dm', 'Am', 'F', 'Am', 'Dm', 'Am', 'F', 'Am', 'Dm', 'Am', 'Bb', 'Bb', 'Bdim', 'Bdim', 'C', 'C', 'Bb', 'Bb', 'Bb', 'Bb', 'Em', 'Bdim', 'Am', 'Em', 'Am', 'Am', 'F', 'F', 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bb', 'C', 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bdim', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'F', 'Bb', 'Em', 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bb', 'C', 'F', 'Am', 'Dm', 'Am', 'F', 'Am', 'Dm', 'Am', 'F', 'Am', 'Dm', 'Am', 'F', 'Am', 'Dm', 'Am', 'Bb', 'Bb', 'Bdim', 'Bdim', 'C', 'C', 'Bb', 'Bb', 'Bb', 'Bb', 'Em', 'Bdim', 'Am', 'Em', 'Am', 'Am', 'F', 'F', 'F', 'Am', 'Dm', 'F', 'Bb', 'Dm', 'Bb', 'C

In [741]:
#一次抓8個
#修改Dominantlist內容，再執行

import csv,os
from operator import itemgetter

txtfilePath = b'E:/downloadmusic4/'
csvfilePath = b'C:/Users/BigData/'
Dominanatvalue = ['G', 'Am', 'Bb', 'C', 'F', 'Dm', 'Em','A']

def Dominantlist_from_txt(txtPath):
    with open(txtPath, 'rb') as txtFile:
        txtreadline = txtFile.readline()
    txtreadline = txtreadline.split(',')
    Dominantlist = []
    for i in range(len(txtreadline)):
        if txtreadline[i] == '0':
            Dominantlist.append(int(txtreadline[i]))
        else:
            Dominantlist.append(txtreadline[i])
    return Dominantlist

def Dominantlist_to_Dominantgram_8freq(Dominantlist):
    return [Dominantlist[i:i+8] for i in range(len(Dominantlist)-7) if (i%2) == 0]

def Dominantgram_to_Dominantdict_8freq(Dominantgram):
    Dominantdict=dict()
    for i in range(len(Dominantgram)):
        if Dominantgram[i][0] != 0 :
            Dominantdict[tuple(Dominantgram[i])] = Dominantdict.get(tuple(Dominantgram[i]),0)+1
    return Dominantdict

def Dominantsorted_count_comparator(x):
    return x[1]

#想像[1~8]，每次往右位移2個，變成[7,8,1~6]，這些可能的變化，都不計算
def Dominantsorted_possible_transfer_clean(Dominantsorted):
    lists = []
    for i in range(len(Dominantsorted)):
        k = 2
        splitcount = 0
        transferlists = []
        for j in range(len(Dominantsorted[0][0])):
            if j%2==0 and j!=0 :
                splitcount += 1
                transferlists.append(Dominantsorted[i][0][(k*(splitcount)):] + Dominantsorted[i][0][:(k*(splitcount))])
        if set(transferlists).isdisjoint(set(lists)) :
            lists.append(Dominantsorted[i][0])
    return lists

def Dominantsorted_classify(Compare_list,Compare_number):
    Compare_list2 = []
    print '用來比對的和弦'
    print Compare_list[Compare_number]
    print '和弦組合當中，類似的組合'
    for i in range(len(Compare_list)):
        Compare_count = 0
        for j in range(len(Compare_list[0])):
            if cmp(Compare_list[Compare_number][j],Compare_list[i][j]) == 0 : Compare_count += 1
        print Compare_count,Compare_list[i]
        if Compare_count >= 6 : Compare_list2.append(Compare_list[i])
    return Compare_list2

#目前一次八個和弦，最多錯兩個和弦，然後去掉重複的，做成list，用來比對
def Dominantsorted_to_Dominantcompare_8freq(Dominantsorted,Compare_number):
    Compare_list = Dominantsorted_possible_transfer_clean(Dominantsorted)
    Compare_Dominant = [Dominantsorted[i] for i in range(len(Dominantsorted)) for j in Compare_list if Dominantsorted[i][0] == j]
    print '可用來比對的和弦組合'
    print Compare_Dominant
    print '總共有幾個和弦組合可用來比對 =',len(Compare_list)
    print '和弦組合當中，類似的組合'
    Compare_list2 = Dominantsorted_classify(Compare_list,Compare_number)
    print '挑選的結果'
    Compare_list3 = [j for j in  Dominantsorted for i in Compare_list2 if j[0] == i]
    print Compare_list3
    print '歸類到最大次數的和弦'
    Compare_list3 = max(Compare_list3,key = Dominantsorted_count_comparator)
    print Compare_list3

for txtfilename in os.listdir(txtfilePath):
    print '檔名 = ' + txtfilename.split('.')[0]
    txtPath = txtfilePath + txtfilename
    Dominantlist = Dominantlist_from_txt(txtPath)
#     print Dominantlist
    Dominantgram_8freq = Dominantlist_to_Dominantgram_8freq(Dominantlist)
#     print Dominantgram_8freq
    Dominantdict_8freq = Dominantgram_to_Dominantdict_8freq(Dominantgram_8freq)
#     print Dominantdict_8freq
    Dominantsorted_8freq=sorted(Dominantdict_8freq.items(), key=itemgetter(1), reverse=True)
#     print Dominantsorted_8freq
# 這裡可以改比對的參數，總共可用來比對的和弦組合，選擇以第幾個和弦，當比對的基準，去比對全部的和弦
    Dominantsorted_to_Dominantcompare_8freq(Dominantsorted_8freq,0)

#寫入csv檔
csvfilePath = b'C:/Users/BigData/8-gram.csv'
# csvPath = csvfilePath + csvfilename + ' 8-gram.csv'

with open(csvfilePath,'wb') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['Dominant', 'Frequency'])
    for i in range(0,len(Dominantsorted_8freq)):
        writer.writerow(Dominantsorted_8freq[i])

可用來比對的和弦組合
[(('Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G'), 20), (('F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm'), 11), (('Dm', 'Dm', 'F', 'F', 'F', 'F', 'A', 'A'), 3), (('G', 'G', 'F', 'F', 'F', 'F', 'C', 'C'), 3), (('C', 'C', 'Dm', 'Dm', 'F', 0, 'F', 0), 2), (('A', 'A', 'A', 0, 'Dm', 'Dm', 'F', 'F'), 2), (('F', 'F', 'F', 'F', 'A', 'A', 'A', 0), 2), (('A', 0, 'Dm', 'Dm', 'F', 'F', 'C', 'C'), 2), (('A', 'A', 0, 0, 0, 0, 0, 0), 1), (('F', 0, 'A', 'A', 'A', 'A', 'A', 'A'), 1), (('A', 'A', 'A', 0, 0, 0, 0, 0), 1), (('A', 'A', 'A', 'A', 0, 0, 0, 0), 1), (('A', 'A', 'A', 'A', 'A', 'A', 'A', 0), 1), (('F', 'F', 'C', 'C', 'Dm', 'Dm', 'F', 0), 1), (('F', 0, 'F', 0, 'A', 'A', 'A', 'A'), 1), (('A', 'A', 'A', 'A', 'A', 0, 0, 0), 1), (('F', 'F', 'F', 'F', 'A', 'A', 'A', 'A'), 1), (('A', 'A', 'A', 'A', 'A', 'A', 0, 0), 1), (('Dm', 'Dm', 'F', 0, 'F', 0, 'A', 'A'), 1), (('A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'), 1), (('F', 'F', 'A', 'A', 'A', 'A', 'A', 'A'), 1), (('A', 0, 0, 0, 0, 0, 0, 0), 1)]
總共有幾個和弦組合可用來比對

In [672]:
#一次抓16個
#修改Dominantlist內容，再執行

from operator import itemgetter
import csv,os

txtfilePath = b'E:/downloadmusic4/'
csvfilePath = b'C:/Users/BigData/'
Dominanatvalue = ['G', 'Am', 'Bb', 'C', 'F', 'Dm', 'Em']

def Dominantlist_from_txt(txtPath):
    with open(txtPath, 'rb') as txtFile:
        txtreadline = txtFile.readline()
    txtreadline = txtreadline.split(',')
    Dominantlist = []
    for i in range(len(txtreadline)):
        if txtreadline[i] == '0':
            Dominantlist.append(int(txtreadline[i]))
        else:
            Dominantlist.append(txtreadline[i])
    return Dominantlist

def Dominantlist_to_Dominantgram_16freq(Dominantlist):
    return [Dominantlist[i:i+16] for i in range(len(Dominantlist)-15) if (i%2) == 0]

def Dominantgram_to_Dominantdict_16freq(Dominantgram):
    Dominantdict=dict()
    for i in range(len(Dominantgram)):
        if Dominantgram[i][0] != 0 :
            Dominantdict[tuple(Dominantgram[i])] = Dominantdict.get(tuple(Dominantgram[i]),0)+1
    return Dominantdict

def Dominantsorted_count_comparator(x):
    return x[1]

#想像[1~16]，每次往右位移2個，變成[15,16,1~14]，這些可能的變化，都不計算
def Dominantsorted_possible_transfer_clean(Dominantsorted):
    lists = []
    for i in range(len(Dominantsorted)):
        k = 2
        splitcount = 0
        transferlists = []
        for j in range(len(Dominantsorted[0][0])):
            if j%2==0 and j!=0 :
                splitcount += 1
                transferlists.append(Dominantsorted[i][0][(k*(splitcount)):] + Dominantsorted[i][0][:(k*(splitcount))])
        if set(transferlists).isdisjoint(set(lists)) :
            lists.append(Dominantsorted[i][0])
    return lists

def Dominantsorted_classify(Compare_list,Compare_number):
    Compare_list2 = []
    print '用來比對的和弦'
    print Compare_list[Compare_number]
    print '和弦組合當中，類似的組合'
    for i in range(len(Compare_list)):
        Compare_count = 0
        for j in range(len(Compare_list[0])):
            if cmp(Compare_list[Compare_number][j],Compare_list[i][j]) == 0 : Compare_count += 1
        print Compare_count,Compare_list[i]
        if Compare_count >= 13 : Compare_list2.append(Compare_list[i])
    return Compare_list2

#目前一次十六個和弦，最多錯三個和弦，然後去掉重複的，做成list，用來比對
def Dominantsorted_to_Dominantcompare_16freq(Dominantsorted,Compare_number):
    Compare_list = Dominantsorted_possible_transfer_clean(Dominantsorted)
    Compare_Dominant = [Dominantsorted[i] for i in range(len(Dominantsorted)) for j in Compare_list if Dominantsorted[i][0] == j]
    print '可用來比對的和弦組合'
    print Compare_Dominant
    print '總共有幾個和弦組合可用來比對 =',len(Compare_list)
    Compare_list2 = Dominantsorted_classify(Compare_list,Compare_number)
    print '挑選的結果'
    Compare_list3 = [j for j in  Dominantsorted for i in Compare_list2 if j[0] == i]
    print Compare_list3
    print '歸類到最大次數的和弦'
    Compare_list3 = max(Compare_list3,key=Dominantsorted_count_comparator)
    print Compare_list3

for txtfilename in os.listdir(txtfilePath):
    print '檔名 = ' + txtfilename.split('.')[0]
    txtPath = txtfilePath + txtfilename
    Dominantlist = Dominantlist_from_txt(txtPath)
#     print Dominantlist
    Dominantgram_16freq = Dominantlist_to_Dominantgram_16freq(Dominantlist)
#     print Dominantgram_16freq
    Dominantdict_16freq = Dominantgram_to_Dominantdict_16freq(Dominantgram_16freq)
#     print Dominantdict_16freq
    Dominantsorted_16freq=sorted(Dominantdict_16freq.items(), key=itemgetter(1), reverse=True)
#     print Dominantsorted_16freq
#這裡可以改比對的參數，總共可用來比對的和弦組合，選擇以第幾個和弦，當比對的基準，去比對全部的和弦
    Dominantsorted_to_Dominantcompare_16freq(Dominantsorted_16freq,1)

#寫入csv檔
csvfilePath = b'C:/Users/BigData/16-gram.csv'
# csvPath = csvfilePath + csvfilename + ' 16-gram.csv'

with open(csvfilePath,'wb') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['Dominant', 'Frequency'])
    for i in range(0,len(Dominantsorted_16freq)):
        writer.writerow(Dominantsorted_16freq[i])

可用來比對的和弦組合
[('Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G', 'Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G'), ('F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm', 'F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm'), ('G', 'G', 'F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm', 'F', 'F', 'F', 'F', 'C', 'C'), ('C', 'C', 'G', 'G', 'Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G', 'F', 'F', 'F', 'F'), ('Dm', 'Dm', 'F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm', 'F', 'F', 'F', 'F', 'A', 'A'), ('Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G', 'F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm'), ('C', 'C', 'Dm', 'Dm', 'F', 'F', 'F', 'F', 'A', 'A', 'A', 0, 'Dm', 'Dm', 'F', 'F'), ('A', 0, 'Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G', 'Dm', 'Dm', 'F', 'F', 'C', 'C'), ('A', 'A', 'A', 0, 'Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G', 'Dm', 'Dm', 'F', 'F'), ('F', 'F', 'F', 'F', 'A', 'A', 'A', 0, 'Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G'), ('F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm', 'F', 'F', 'F', 'F', 'A', 'A', 'A', 0), ('F', 'F', 'F', 'F', 'C', 'C', 'Dm', 'Dm', 'F', 'F', 'F', 'F

In [38]:
#測試區
series=(0,1,2,3,4)
sqr_series=[num**2 for num in series]
print(sqr_series)

[0, 1, 4, 9, 16]
['\xe5', '\x90', '\x83', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe4', '\xb8', '\x8d', '\xe5', '\x90', '\x90', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe7', '\x9a', '\xae', '\xef', '\xbc', '\x8c', '\xe4', '\xb8', '\x8d', '\xe5', '\x90', '\x83', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe5', '\x80', '\x92', '\xe5', '\x90', '\x90', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe7', '\x9a', '\xae', '\xe3', '\x80', '\x82']
{'\xa1': 4, '\x80': 2, '\x83': 2, '\x82': 1, '\xe5': 5, '\x84': 4, '\xe7': 2, '\xe3': 1, '\xe8': 8, '\x8d': 2, '\x8c': 1, '\xef': 1, '\xae': 2, '\x91': 4, '\x90': 10, '\x92': 1, '\xb8': 2, '\x9a': 2, '\xbc': 1, '\xe4': 2}
[('\x90', 10), ('\xe8', 8), ('\xe5', 5), ('\xa1', 4), ('\x84', 4), ('\x91', 4), ('\x80', 2), ('\x83', 2), ('\xe7', 2), ('\x8d', 2), ('\xae', 2), ('\xb8', 2), ('\x9a', 2), ('\xe4', 2), ('\x82', 1), ('\xe3', 1), ('\x8c', 1), ('\xef', 1), ('\x92', 1), ('\xbc', 1)]


In [143]:
#測試區
def list2freqdict(mylist):
    mydict=dict()
    for ch in mylist:
        mydict[ch]=mydict.get(ch,0)+1
    return mydict


sentence='吃葡萄不吐葡萄皮，不吃葡萄倒吐葡萄皮。'
chlist=[ch for ch in sentence]
print chlist


chfreqdict=list2freqdict(chlist)
print chfreqdict


from operator import itemgetter
chfreqsorted=sorted(chfreqdict.items(), key=itemgetter(1), reverse=True)
print chfreqsorted


chfreqsorted2=chfreqsorted[:5]

chfreqsorted3=list()
for (ch,num) in chfreqsorted:
    if num > 1:
        chfreqsorted3.append((ch,num))    

print(chfreqsorted2)
print(chfreqsorted3)

#兩字，三字
def list2bigram(mylist):
    return [mylist[i:i+2] for i in range(0,len(mylist)-1)]

def list2trigram(mylist):
    return [mylist[i:i+3] for i in range(0,len(mylist)-2)]

chbigram=list2bigram(chlist)
chtrigram=list2trigram(chlist)
print(chbigram)
print(chtrigram)

def bigram2freqdict(mybigram):
    mydict=dict()
    for (for i in range(0,len(mybigram))) in mybigram:
        mydict[(ch1)]=mydict.get((ch1,ch2),0)+1
    return mydict

def trigram2freqdict(mytrigram):
    mydict=dict()
    for (ch1,ch2,ch3) in mytrigram:
        mydict[(ch1,ch2,ch3)]=mydict.get((ch1,ch2,ch3),0)+1
    return mydict

bigramfreqdict=bigram2freqdict(chbigram)
trigramfreqdict=trigram2freqdict(chtrigram)
print(bigramfreqdict)
print(trigramfreqdict)


bigramfreqsorted=sorted(bigramfreqdict.items(), key=itemgetter(1), reverse=True)
trigramfreqsorted=sorted(trigramfreqdict.items(), key=itemgetter(1), reverse=True)
print(bigramfreqsorted[:5])
print(trigramfreqsorted[:5])

def freq2report(freqlist):
    chs=str()
    print('Char(s)\tCount')
    print('=============')
    for (token,num) in freqlist:
        for ch in token:
            chs=chs+ch
        print(chs,'\t',num)
        chs=''
    return

freq2report(chfreqsorted)
freq2report(bigramfreqsorted)
freq2report(trigramfreqsorted)

SyntaxError: invalid syntax (<ipython-input-143-b00ef5819c4a>, line 47)

In [45]:
#測試區
aaa = '\xe5\x90\x83\xe8\x91\xa1\xe8\x90\x84\xe4\xb8\x8d\xe5\x90\x90\xe8\x91\xa1\xe8\x90\x84\xe7\x9a\xae\xef\xbc\x8c\xe4\xb8\x8d\xe5\x90\x83\xe8\x91\xa1\xe8\x90\x84\xe5\x80\x92\xe5\x90\x90\xe8\x91\xa1\xe8\x90\x84\xe7\x9a\xae\xe3\x80\x82' # It mean '單車'
print aaa.decode('unicode-escape')
p = (aaa.decode('unicode-escape'))
print type(p)
print type(aaa)

åè¡èä¸åè¡èç®ï¼ä¸åè¡èååè¡èç®ã
<type 'unicode'>
<type 'str'>


In [50]:
#測試區
sentence='吃葡萄不吐葡萄皮，不吃葡萄倒吐葡萄皮。'
chlist=[ch for ch in sentence]
print chlist
print len(chlist)

['\xe5', '\x90', '\x83', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe4', '\xb8', '\x8d', '\xe5', '\x90', '\x90', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe7', '\x9a', '\xae', '\xef', '\xbc', '\x8c', '\xe4', '\xb8', '\x8d', '\xe5', '\x90', '\x83', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe5', '\x80', '\x92', '\xe5', '\x90', '\x90', '\xe8', '\x91', '\xa1', '\xe8', '\x90', '\x84', '\xe7', '\x9a', '\xae', '\xe3', '\x80', '\x82']
57
